In [1]:
import numpy as np

data=np.load('data.npy')
target=np.load('target.npy')

In [2]:
import tensorflow
from tensorflow import keras
%env SM_FRAMEWORK=tensorflow.keras

from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D,Activation,MaxPooling2D
from tensorflow.keras.utils import normalize
from tensorflow.keras.layers import Concatenate
from tensorflow.keras import Input
from tensorflow.keras.callbacks import ModelCheckpoint

input_shape=data.shape[1:] 
inp=Input(shape=input_shape)
convs=[]

parrallel_kernels=[3,5,7]

for k in range(len(parrallel_kernels)):

    conv = Conv2D(128, parrallel_kernels[k],padding='same',activation='relu',input_shape=input_shape,strides=1)(inp)
    convs.append(conv)

out = Concatenate()(convs)
conv_model = Model(inp,out)

model = Sequential()
model.add(conv_model)

model.add(Conv2D(64,(3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(32,(3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(128,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(2,input_dim=128,activation='softmax'))
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

model.summary()

env: SM_FRAMEWORK=tensorflow.keras
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
model (Functional)           (None, 10, 10, 384)       11008     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 8, 8, 64)          221248    
_________________________________________________________________
activation (Activation)      (None, 8, 8, 64)          0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 4, 4, 64)          0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 2, 2, 32)          18464     
_________________________________________________________________
activation_1 (Activation)    (None, 2, 2, 32)          0         
_________________________________________________________________
max_pooling2d_1 (MaxP

In [3]:
import sklearn
from sklearn.model_selection import train_test_split

train_data,test_data,train_target,test_target=train_test_split(data,target,test_size=0.1)

In [4]:
checkpoint = ModelCheckpoint('model-{epoch:03d}.model',monitor='val_loss',verbose=0,save_best_only=True,mode='auto')
history=model.fit(train_data,train_target,epochs=20,callbacks=[checkpoint],validation_split=0.1)

Epoch 1/20
6/6 [==============================] - 7s 421ms/step - loss: 0.6934 - accuracy: 0.4444 - val_loss: 0.6934 - val_accuracy: 0.4444
INFO:tensorflow:Assets written to: model-001.model\assets
Epoch 2/20
6/6 [==============================] - 2s 332ms/step - loss: 0.6927 - accuracy: 0.5617 - val_loss: 0.6937 - val_accuracy: 0.4444
Epoch 3/20
6/6 [==============================] - 2s 319ms/step - loss: 0.6929 - accuracy: 0.5309 - val_loss: 0.6932 - val_accuracy: 0.4444
INFO:tensorflow:Assets written to: model-003.model\assets
Epoch 4/20
6/6 [==============================] - 2s 328ms/step - loss: 0.6934 - accuracy: 0.5123 - val_loss: 0.6928 - val_accuracy: 0.5556
INFO:tensorflow:Assets written to: model-004.model\assets
Epoch 5/20
6/6 [==============================] - 2s 320ms/step - loss: 0.6934 - accuracy: 0.5062 - val_loss: 0.6926 - val_accuracy: 0.5556
INFO:tensorflow:Assets written to: model-005.model\assets
Epoch 6/20
6/6 [==============================] - 2s 310ms/step - lo

In [ ]:
from matplotlib import pyplot as plt

plt.plot(history.history['loss'],'r',label='training loss')
plt.plot(history.history['val_loss'],label='validation loss')
plt.xlabel('# epochs')
plt.ylabel('loss')
plt.legend()
plt.show()

In [ ]:
plt.plot(history.history['accuracy'],'r',label='training accuracy')
plt.plot(history.history['val_accuracy'],label='validation accuracy')
plt.xlabel('# epochs')
plt.ylabel('loss')
plt.legend()
plt.show()

In [5]:
print(model.evaluate(test_data,test_target))

1/1 [==============================] - 0s 83ms/step - loss: 0.6946 - accuracy: 0.4500
[0.6946083903312683, 0.44999998807907104]
